In [1]:
print("hello world")

hello world


In [1]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite"
output_path = "detector.tflite"

response = requests.get(url)
if response.status_code == 200:
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded to {output_path}")
else:
    print(f"Failed to download. HTTP Status Code: {response.status_code}")



Downloaded to detector.tflite


In [3]:
%pip install mediapipe

  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.0
    Uninstalling numpy-1.21.0:
      Successfully uninstalled numpy-1.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.10.0 requires protobuf<4,>=3.12, but you have protobuf 4.25.5 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 4.25.5 which is incompatible.
tensorflow 2.5.0 requires flatbuffers~=1.12.0, but you have flatbuffers 25.1.21 which is incompatible.
tensorflow 2.5.0 requires grpcio~=1.34.0, but you have grpcio 1.69.0 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.26.4 which is incompatible.
tensorflow 2.5.0 requires typing-extensions~=

In [8]:
%pip install "protobuf>=3.20.0,<3.21.0"

  Using cached protobuf-3.20.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp39-cp39-macosx_10_9_x86_64.whl (982 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.69.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
mediapipe 0.10.20 requires protobuf<5,>=4.25.3, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.5.0 requires flatbuffers~=1.12.0, but you have flatbuffers 25.1.21 which is incompatible.
tensorflow 2.5.0 requires grpcio~=1.34.0, but you have grpcio 1.69.0 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.26.4 which is incompatible.
tensorflow 2.

In [2]:
import cv2
import mediapipe as mp
import time

# Function to fetch the direct video URL using yt-dlp
def get_video_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'format': 'best[ext=mp4]'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        return info['url']

# Initialize MediaPipe solutions
mp_face_detection = mp.solutions.face_detection
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize Face Detection and Pose Detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.9)
pose = mp_pose.Pose(min_detection_confidence=0.8)

profiles = {}

# Function to assign or update ID based on face embedding and spatial proximity
def assign_id(embedding, bbox, profiles, threshold=0.6, spatial_threshold=50):
    best_match_id = None
    best_similarity = float("inf")
    best_distance = float("inf")

    bbox_center = ((bbox[0] + bbox[2] / 2), (bbox[1] + bbox[3] / 2))

    for profile_id, profile in profiles.items():
        # Compare embedding similarity
        similarity = np.linalg.norm(np.array(profile['face_embedding']) - np.array(embedding))
        # Compare spatial proximity
        profile_bbox = profile["bbox"]
        profile_center = ((profile_bbox[0] + profile_bbox[2] / 2), (profile_bbox[1] + profile_bbox[3] / 2))
        spatial_distance = euclidean(profile_center, bbox_center)

        # Find the best match
        if similarity < threshold and spatial_distance < spatial_threshold:
            if similarity < best_similarity or (similarity == best_similarity and spatial_distance < best_distance):
                best_match_id = profile_id
                best_similarity = similarity
                best_distance = spatial_distance

    if best_match_id is not None:
        # Update the matched profile
        profiles[best_match_id]["bbox"] = bbox
        profiles[best_match_id]["last_seen"] = frame_count
        return best_match_id
    else:
        # Create a new profile
        new_id = len(profiles) + 1
        profiles[new_id] = {"face_embedding": embedding, "bbox": bbox, "last_seen": frame_count}
        return new_id


# YouTube video URL
youtube_url = "https://www.youtube.com/watch?v=9D0WmzB5ovY"  # Replace with your video URL

# Process a YouTube video
youtube_url = "https://www.youtube.com/watch?v=96Y6mc3C1Bg"  # Replace with your URL
video_url = get_video_url(youtube_url)
print("Video URL:", video_url)

cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Unable to open the YouTube video stream.")
else:
    print("Processing YouTube video. Press 'q' to quit.")


    # Specify the start time (in seconds)
    start_time = 320  # Start the video at 30 seconds
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)  # Jump to the start time


    # Adjustable frame rate (delay in milliseconds)
    frame_delay_ms = 100  # Adjust this value to control the frame rate

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading the video stream.")
            break

        # Skip frames for efficiency
        if frame_count % 5 != 0:
            frame_count += 1
            continue

        # Validate frame dimensions
        if frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
            print(f"Skipped frame {frame_count}: invalid dimensions.")
            continue

        # Get frame dimensions
        ih, iw, _ = frame.shape

        # Convert frame to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Face Detection
        face_results = face_detection.process(rgb_frame)
        face_bboxes = []

        if face_results.detections:
            for i, detection in enumerate(face_results.detections):
                # Extract bounding box
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                              int(bboxC.width * iw), int(bboxC.height * ih))
                face_bboxes.append((x, y, w, h))

                # Draw bounding box around the face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Add label for each face
                cv2.putText(frame, f"Face {i+1}", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Pose Detection
        pose_results = pose.process(rgb_frame)
        if pose_results.pose_landmarks:
            filtered_landmarks = []
            for i, lm in enumerate(pose_results.pose_landmarks.landmark):
                x, y = int(lm.x * iw), int(lm.y * ih)
                inside_face = any(fx <= x <= fx + fw and fy <= y <= fy + fh for fx, fy, fw, fh in face_bboxes)
                if not inside_face:  # Only include landmarks outside face bounding boxes
                    filtered_landmarks.append((x, y))

            # Draw filtered pose landmarks
            for x, y in filtered_landmarks:
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)


        # Display the processed frame
        cv2.imshow("MediaPipe YouTube Video Processing", frame)

        # Adjustable frame delay
        time.sleep(frame_delay_ms / 1000.0)  # Convert ms to seconds

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting video processing.")
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


AttributeError: module 'numpy' has no attribute 'typeDict'

/opt/miniconda3/envs/office/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:511: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [4]:
import yt_dlp
import cv2
import mediapipe as mp
import time
import numpy as np
from deepface import DeepFace
from scipy.spatial.distance import euclidean

# Function to fetch the direct video URL using yt-dlp
def get_video_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'format': 'best[ext=mp4]'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        return info['url']

# Initialize MediaPipe solutions
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Initialize Face Detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Profiles for tracking people
profiles = {}
max_inactive_frames = 30  # Maximum frames to keep a profile if not detected

# Function to assign or update ID based on face embedding and spatial proximity
def assign_id(embedding, bbox, profiles, frame_count, threshold=0.6, spatial_threshold=50):
    best_match_id = None
    best_similarity = float("inf")
    best_distance = float("inf")

    bbox_center = ((bbox[0] + bbox[2] / 2), (bbox[1] + bbox[3] / 2))

    for profile_id, profile in profiles.items():
        # Compare embedding similarity
        similarity = np.linalg.norm(np.array(profile['face_embedding']) - np.array(embedding))
        # Compare spatial proximity
        profile_bbox = profile["bbox"]
        profile_center = ((profile_bbox[0] + profile_bbox[2] / 2), (profile_bbox[1] + profile_bbox[3] / 2))
        spatial_distance = euclidean(profile_center, bbox_center)

        # Find the best match
        if similarity < threshold and spatial_distance < spatial_threshold:
            if similarity < best_similarity or (similarity == best_similarity and spatial_distance < best_distance):
                best_match_id = profile_id
                best_similarity = similarity
                best_distance = spatial_distance

    if best_match_id is not None:
        # Update the matched profile
        profiles[best_match_id]["bbox"] = bbox
        profiles[best_match_id]["last_seen"] = frame_count
        return best_match_id
    else:
        # Create a new profile
        new_id = len(profiles) + 1
        profiles[new_id] = {"face_embedding": embedding, "bbox": bbox, "last_seen": frame_count}
        return new_id

# YouTube video URL
youtube_url = "https://www.youtube.com/watch?v=9D0WmzB5ovY"  # Replace with your video URL

# Fetch the video stream URL
video_url = get_video_url(youtube_url)
print("Video URL:", video_url)

# Open the video stream with OpenCV
cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Unable to open the YouTube video stream.")
else:
    print("Processing YouTube video. Press 'q' to quit.")

    # Specify the start time (in seconds)
    start_time = 30  # Start the video at 30 seconds
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)  # Jump to the start time

    # Adjustable frame rate (delay in milliseconds)
    frame_delay_ms = 100  # Adjust this value to control the frame rate
    frame_count = 0  # Initialize frame counter

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading the video stream.")
            break

        # Increment frame count
        frame_count += 1

        # Convert the frame to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the frame
        face_results = face_detection.process(rgb_frame)
        if face_results.detections:
            for i, detection in enumerate(face_results.detections):
                # Extract bounding box
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                              int(bboxC.width * iw), int(bboxC.height * ih))
                bbox = (x, y, w, h)

                # Extract face region
                face_roi = rgb_frame[y:y+h, x:x+w]
                if face_roi.size == 0:
                    continue

                try:
                    # Generate face embedding
                    embedding = DeepFace.represent(face_roi, model_name="Facenet", enforce_detection=False)[0]['embedding']
                    person_id = assign_id(embedding, bbox, profiles, frame_count)

                    # Draw bounding box and label with embedding ID
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID: {person_id}", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                except Exception as e:
                    print(f"Error processing face embedding in frame {frame_count}: {e}")

        # Remove stale profiles
        profiles = {k: v for k, v in profiles.items() if frame_count - v["last_seen"] <= max_inactive_frames}

        # Display the processed frame
        cv2.imshow("MediaPipe YouTube Video Processing", frame)

        # Adjustable frame delay
        time.sleep(frame_delay_ms / 1000.0)  # Convert ms to seconds

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting video processing.")
            break

cap.release()
cv2.destroyAllWindows()

# Save profiles to JSON
with open("profiles.json", "w") as f:
    json.dump(profiles, f, indent=4)
print(f"Profiles saved to profiles.json")


I0000 00:00:1734596265.413802 3508182 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 555 OpenGL Engine
W0000 00:00:1734596265.419068 3511817 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video URL: https://rr1---sn-jxopj-n5oe.googlevideo.com/videoplayback?expire=1734617868&ei=rNZjZ4y3A4yWsfIPyr3w2QI&ip=2607%3Af140%3A400%3A6b%3A8481%3Aaa15%3A119c%3Ac39f&id=o-AFtpSAoHUUj2X_nEuvUDEBJTYDYOW6ZtWTWpFkaX3dDc&itag=18&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1734596268%2C&mh=KA&mm=31%2C29&mn=sn-jxopj-n5oe%2Csn-o097znse&ms=au%2Crdu&mv=m&mvi=1&pl=32&rms=au%2Cau&initcwndbps=4312500&bui=AfMhrI8OH-KxSNZ0tDqUW_7UVK_xXBKcBpaWy6FGqrSoAmBVQXmcIerS02jQxVQNL2Q-4bORx6Ok_5wH&vprv=1&svpuc=1&mime=video%2Fmp4&ns=0wKDSEkWV9J-LPo8ZdaX_2MQ&rqh=1&gir=yes&clen=124173805&ratebypass=yes&dur=2031.931&lmt=1733188644547912&mt=1734596001&fvip=2&fexp=51326932%2C51331020%2C51335594%2C51371293&c=MWEB&sefc=1&txp=4438434&n=O3wOY7x5bLucgQ&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&sig=AJfQdSswRQIgdx9x9lNA9o5W4EyaISxwiVh1r2BYkEp2wNZSk7CiY3kCIQCdv9yLG42oqt9bTNgYNJgQX5iQVrNd1NTC0H-bni661Q%3D%3D&lsparam

NameError: name 'json' is not defined

In [ ]:
#old code that just streams without mediapipe facial recognition
# Get the direct video URL
import yt_dlp
import cv2

def get_video_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'format': 'best[ext=mp4]'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        return info['url']

# Get the direct video URL
youtube_url = "https://www.youtube.com/watch?v=m34ZKJNyxac"
video_url = get_video_url(youtube_url)
print("Video URL:", video_url)

# Open video stream with OpenCV
cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Unable to open video stream.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("YouTube Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
video_url = get_video_url(youtube_url)
print("Video URL:", video_url)

# Open video stream with OpenCV
cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Unable to open video stream.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("YouTube Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Video URL: https://rr2---sn-o097znsr.googlevideo.com/videoplayback?expire=1733935790&ei=Tm5ZZ8OCHvr-sfIP1OPhoAg&ip=2601%3A644%3A601%3Afa0%3A18f1%3A6d9c%3A9455%3Ae968&id=o-AM0VWy9id_RhIja_CuAX_7jCOsmuB3nF0M_O12ZHnnTH&itag=18&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1733914190%2C&mh=Vq&mm=31%2C29&mn=sn-o097znsr%2Csn-n4v7sns7&ms=au%2Crdu&mv=m&mvi=2&pl=34&rms=au%2Cau&initcwndbps=3915000&bui=AQn3pFQKhi7MVAR6il70JWVEwJTRLPZJ7nzHT55pqdE6sdGYDy3fT88Js2ixpJEj8o83imVFwTnhby9Q&vprv=1&svpuc=1&mime=video%2Fmp4&ns=dgnEsjCi-frwcKg6mSObwzwQ&rqh=1&gir=yes&clen=36612911&ratebypass=yes&dur=398.593&lmt=1733879672190924&mt=1733913906&fvip=3&fexp=51326932%2C51331020%2C51335594%2C51347747&c=MWEB&sefc=1&txp=5538434&n=jbygYr9NxynRtw&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&sig=AJfQdSswRQIgR_sLg4GdoV2OPdKEWhxKh-0u7jr7-NyGlLR3-mI-5a4CIQCMCLTi1P8C5kClsDZoxqTwkw4zeYhfluOoJaAbPTohlA%3D%3D&lsparams=met%

KeyboardInterrupt: 

: 

In [ ]:
#Todo, need to add cookies for age restricted content